# ManyCores exercises

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from matplotlib import rcParams
import itertools
%matplotlib inline

from IPython.display import display

In [ ]:
def csv_parse(path, header=False, sep=";"):
    if header:
        df = pd.read_csv(path, sep=sep, names=header)
    else:
        df = pd.read_csv(path, sep=sep)
    #display(df.head())
    return df

def load_glob(glob, concat=True):
    liste = [pd.read_csv(file) for file in glob]
    name = [file for file in glob]
    repr(name)
    if concat:
        return pd.concat(liste)
    return liste, name

In [ ]:
def transform_to_ms(df, cols):
    return df[cols]*1e3

def bandwidth(df, cols):
    for x in cols:
        df["bw_"+x] = 2 * df["N"]*df["N"]* 8 / df[x] * 1e-9
    return df

# Ex9.1

In [ ]:
# Get the data
cwd = pathlib.Path(os.getcwd())
data_subdir = cwd.parent / "data"
plot_dir = cwd / "out"
data_subdir, plot_dir

In [ ]:
def drop_cols(df):
    to_drop = [x for x in df.columns if x not in ["N", "runtime"]]
    return df.drop(to_drop, axis="columns")


In [ ]:
df_omp = csv_parse(data_subdir / "ph_data_omp.csv")
ks = df_omp["k"]

df_ocl = csv_parse(data_subdir / "ph_data_mykernel_ocl.csv")
df_ocl = df_ocl.rename(columns={"dot_time": "runtime"})
df_ocl = drop_cols(df_ocl)
df_ocl.insert(1, column="k", value=ks)

df_cuda = csv_parse(data_subdir / "ph_data_mykernel_cuda.csv")
df_cuda = df_cuda.rename(columns={"dot_time": "runtime"})
df_cuda = drop_cols(df_cuda)
df_cuda.insert(1, column="k", value=ks)

df = df_omp.rename(columns={"runtime": "OMP"})
df = df.join(df_ocl["runtime"]).rename(columns={"runtime": "OpenCL"})
df = df.join(df_cuda["runtime"]).rename(columns={"runtime": "CUDA"})
df_runtime = df
del df
df_runtime

In [ ]:
df_speedup = df_runtime.copy()
df_speedup["OpenCL"] = df_speedup["OMP"] / df_speedup["OpenCL"]
df_speedup["CUDA"] = df_speedup["OMP"] / df_speedup["CUDA"] 
df_speedup

In [ ]:
def print_styling():

    plt.rc('figure', figsize=(12,8)) 
    plt.style.use("seaborn")
    #sns.set_palette("Paired")
    SMALL_SIZE = 15
    MEDIUM_SIZE = 18
    BIGGER_SIZE = 26

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('lines', linewidth=2)

    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
print_styling()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
print_styling()
axs = axs.ravel()
axL = axs[0]
axR = axs[1]

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

df_runtime.plot(x="N", y=["OMP", "OpenCL", "CUDA"], ax=axL, logx=logx, logy=logy)
df_speedup.plot(x="N", y=["OpenCL", "CUDA"], ax=axR, logx=logx, logy=logy)

axL.set_ylabel("runtime [s]")

axL.set_title("Runtime")
axR.set_title("Speedup")

plt.suptitle("OMP vs OpenCL vs CUDA")
plt.tight_layout()
fig.savefig(plot_dir / "omp_vs_cuda_vs_opencl.pdf", bbox_inches="tight", pad_inches=0.2)

# Task 2: Transpose

In [ ]:
df = csv_parse(data_subdir / "ph_data_transpose.csv")
df_speedup = df[["N", "naive", "my_block", "nv_block"]]
rt_cols = ["naive", "my_block", "nv_block"]

for version in ["my_block", "nv_block"]:
    df_speedup[version] = df_speedup[version] / df_speedup["naive"]
df = bandwidth(df, ["naive", "my_block", "nv_block"])
df_bw = df[["N", "bw_naive", "bw_my_block", "bw_nv_block"]]
bw_cols = ["bw_naive", "bw_my_block", "bw_nv_block"]

df

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12,8))
print_styling()
axs = axs.ravel()
axL = axs[0]
axR = axs[1]

logx, logy = True, True
if (logx and logy):
    plot_format = "loglog"
if (logx and not logy):
    plot_format = "logx"
if (not logx and logy):
    plot_format = "logy"
else:
    plot_format = "linear"

df.plot(x="N", y=rt_cols, ax=axL, logx=logx, logy=logy, marker="o")
df_bw.plot(x="N", y=bw_cols, ax=axR, logx=logx, logy=logy, marker="o")
#df_speedup.plot(x="N", y=["my_block", "nv_block"], ax=axR, logx=logx, logy=logy)

axL.set_xscale("log", base=2)
axL.set_xticklabels([0,1,512, 1024, 2048, 4096, 8192])
axR.set_xscale("log", base=2)
axR.set_xticklabels([0,1,512, 1024, 2048, 4096, 8192])

axL.set_ylabel("runtime [s]")
axR.set_ylabel("bandwith [GB/s]")
axR.set_yticklabels([1,2,10, 100])

axL.set_title("Runtime")
axR.set_title("Bandwidth")

axR.legend(rt_cols)

plt.suptitle("Transpose")
plt.tight_layout()
fig.savefig(plot_dir / "transpose.pdf", bbox_inches="tight", pad_inches=0.2)